In [4]:
import re
from typing import List, Dict, NamedTuple, Optional

In [7]:
class Bag(NamedTuple):
    bag_type: str
    num: int = 1
           
    @staticmethod
    def parse_text(text:str):
        """
        3 pale aqua bag or duffy douche bag
        """
        REGEX = "^ ?(?P<num>[0-9])? (?P<type>[a-z]+ [a-z]+) bags"
        m = re.match(REGEX, text)
        return Bag(num = int(m.group('num')), 
                   bag_type = m.group('type'))          


class Node(NamedTuple):
    parent: Bag
    child: Dict[str, int]
        
    @staticmethod
    def parse_row(row:str)-> Bag:
        """
        handle input row day 6
        """
        contain = row.split("contain")
        contain0 = "1 " + contain[0] # not so clean...
        commas = contain[1].split(",")
        if len(commas) == 1:
            return Node(main = Bag.parse_text(contain0))
        else:
            inside = [
                Bag.parse_text(t)
                for t in commas
            ]
        main = Bag.parse_text(contain0)
        return Node(main= main, inside=inside)


In [8]:
TEST = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [9]:
RAW1 = "clear crimson bags contain 3 pale aqua bags, 4 plaid magenta bags, 3 dotted beige bags, 3 dotted black bags."
RAW2 = "shiny plum bags contain no other bags."

In [10]:
ROWS = TEST.split("\n")

In [11]:
test = [Node.parse_row(r) 
 for r in ROWS]

AttributeError: 'NoneType' object has no attribute 'group'

In [81]:
ds = {n.main.btype:n.inside
      for n in test}

In [87]:
target = 'shiny gold'
union = set(ds[target])
for btype, inside in ds.items():
    print(btype, inside)
    if inside and target in inside:
        for bag in inside:
            union |= set(ds[bag])

light red [Bag(btype='bright white', num=1), Bag(btype='muted yellow', num=2)]
dark orange [Bag(btype='bright white', num=3), Bag(btype='muted yellow', num=4)]
bright white None
muted yellow [Bag(btype='shiny gold', num=2), Bag(btype='faded blue', num=9)]
shiny gold [Bag(btype='dark olive', num=1), Bag(btype='vibrant plum', num=2)]
dark olive [Bag(btype='faded blue', num=3), Bag(btype='dotted black', num=4)]
vibrant plum [Bag(btype='faded blue', num=5), Bag(btype='dotted black', num=6)]
faded blue None
dotted black None
